In [1]:
import numpy as np
from keras.layers import (BatchNormalization, Conv1D, Dense,
                          Input, GRU, TimeDistributed, Activation, Lambda)
from keras.models import Model
from data_generator import DataGenerator
from keras import backend as K
from utils import calc_feat_dim, spectrogram_from_file, text_to_int_sequence

Using TensorFlow backend.


In [2]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def train(input_dim=161, output_dim=29, recur_layers=1, filters=1024, 
          kernel_size=11, conv_border_mode='valid', conv_stride=2, 
          initialization='glorot_uniform', batch_norm=True, batch_size=30):
    
    # call to data generator
    audio_gen = AudioGenerator()
   
    # define the model
    input_data = Input(name='the_input', shape=(None, input_dim))
    conv_1d = Conv1D(filters, kernel_size, name='conv1d',
                     padding=conv_border_mode,
                     strides=conv_stride, 
                     kernel_initializer=initialization,
                     activation='relu')(input_data)
    output = BatchNormalization(name='bn_conv_1d')(conv_1d)
    for r in range(recur_layers):
        output = GRU(filters, activation='relu',
                     name='rnn_{}'.format(r + 1), kernel_initializer=initialization,
                     return_sequences=True)(output)
        bn_layer = BatchNormalization(name='bn_rnn_{}'.format(r + 1))
        output = bn_layer(output)

    # transform NN output to character activations
    network_output = TimeDistributed(Dense(
        output_dim, name='dense', kernel_initializer=initialization))(output)
    Model(inputs=input_data, outputs=network_output).summary()
    
    labels = Input(name='the_labels', shape=(199,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([network_output, labels, input_length, label_length])
    
    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
    
    # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='sgd')
        
    model.fit_generator(generator=next_batch(audio_gen, mb_size = batch_size),
                        steps_per_epoch=100, #2700//batch_size,
                        epochs=1, verbose=1)

In [3]:
def AudioGenerator(mb_size=30):
    # prepare the data generator
    datagen = DataGenerator()
    # load the JSON file that contains the dataset
    datagen.load_train_data('train_corpus.json')
    # Use a few samples from the dataset, to calculate the means and variance
    # of the features, so that we can center our inputs to the network
    datagen.fit_train(100)
    return datagen
    
def next_batch(datagen, mb_size = 30):
    # get all features for all audio paths
    audio_paths = datagen.train_audio_paths
    features = [featurize(datagen, a) for a in audio_paths]
    
    # get max audio length
    max_aud_length = max([f.shape[0] for f in features])
    max_lab_length = max([len(datagen.train_texts[i]) for i in range(len(datagen.train_texts))])
    
    # Pad all the inputs so that they are all the same length
    x = np.zeros([mb_size, max_aud_length, features[0].shape[1]])
    y = np.ones([mb_size, max_lab_length]) * -1
    input_length = np.zeros([mb_size,1])
    label_length = np.zeros([mb_size,1])
    while True:
        for i in range(mb_size):
            # x
            feat = normalize(datagen, features[i])  # Center using means and std
            x[i, :feat.shape[0], :] = feat

            # y, input_length, label_length
            label = np.array(text_to_int_sequence(datagen.train_texts[i]))
            y[i, :len(label)] = label
            input_length[i] = features[i].shape[0]
            label_length[i] = len(label)
        outputs = {'ctc': np.zeros([mb_size])}
        inputs = {'the_input': x, # array; dim: mb_size x max_aud_length x features[0].shape[1]
                  'the_labels': y, # array; dim: mb_size, time_steps, num_categories
                  'input_length': input_length, # array; dim: mb_size x 1
                  'label_length': label_length # array; dim: mb_size x 1
                 }
        yield (inputs, outputs)
        x = np.zeros([mb_size, max_aud_length, features[0].shape[1]])
        y = np.ones([mb_size, max_lab_length]) * -1
        input_length = np.zeros([mb_size,1])
        label_length = np.zeros([mb_size,1])

def featurize(generator, audio_clip):
        """ For a given audio clip, calculate the log of its Fourier Transform
        Params:
            generator: 
            audio_clip(str): Path to the audio clip
        """
        return spectrogram_from_file(audio_clip, 
                                     step=generator.step, 
                                     window=generator.window, 
                                     max_freq=generator.max_freq)
    
def normalize(generator, feature, eps=1e-14):
    return (feature - generator.feats_mean) / (generator.feats_std + eps)

In [4]:
train()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, None, 161)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 1024)        1814528   
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 1024)        4096      
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 1024)        6294528   
_________________________________________________________________
bn_rnn_1 (BatchNormalization (None, None, 1024)        4096      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 29)          29725     
Total params: 8,146,973
Trainable params: 8,142,877
Non-trainable params: 4,096
______________________________________________________________

InvalidArgumentError: sequence_length(0) <= 493
	 [[Node: ctc/CTCLoss = CTCLoss[ctc_merge_repeated=true, preprocess_collapse_repeated=false, _device="/job:localhost/replica:0/task:0/cpu:0"](ctc/Log, ctc/ToInt64, ctc/ToInt32_2, ctc/ToInt32_1)]]

Caused by op 'ctc/CTCLoss', defined at:
  File "/Users/alexis/anaconda3/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/alexis/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/alexis/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/alexis/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/alexis/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/alexis/anaconda3/lib/python3.5/site-packages/zmq/eventloop/iolo